# Sistem Rekomendasi Artikel Menggunakan Content-Based Filtering
# 1. Pendahuluan
Notebook ini berisi implementasi sistem rekomendasi artikel menggunakan pendekatan Content-Based Filtering.
Pendekatan ini bekerja dengan cara merekomendasikan artikel yang memiliki kemiripan konten dengan artikel yang telah dibaca pengguna.
Dataset diambil dari Kaggle: https://www.kaggle.com/datasets/jainilcoder/article-recommendation-system

# 2. Import Library
Tahap ini mengimpor seluruh pustaka yang dibutuhkan, seperti pandas untuk manipulasi data,
nltk untuk preprocessing teks, dan scikit-learn untuk vektorisasi serta penghitungan kemiripan antar dokumen.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re


## Download NLTK
Mendownload korpus "stopwords" dari library NLTK

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# 3. Load Dataset
Memuat Dataset Artikel, Struktur dataset terdiri dari dua kolom utama:

1. `'Titles'`

2. `'Article'.`

In [8]:
df = pd.read_csv('articles.csv', encoding='latin1')
df.head()

,Article,Title
0,Data analysis is the process of inspecting and...,Best Books to Learn Data Analysis
1,The performance of a machine learning algorith...,Assumptions of Machine Learning Algorithms
2,You must have seen the news divided into categ...,News Classification with Machine Learning
3,When there are only two classes in a classific...,Multiclass Classification Algorithms in Machin...
4,The Multinomial Naive Bayes is one of the vari...,Multinomial Naive Bayes in Machine Learning


# 4. Explatory Data Analysis (EDA)

## Melihat Informasi Data
 Menampilkan ringkasan informasi DataFrame, termasuk tipe data dan jumlah nilai non-null.

In [9]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Article  34 non-null     object
 1   Title    34 non-null     object
dtypes: object(2)
memory usage: 676.0+ bytes
None


## Menangani Missing Values
Menghitung dan menampilkan jumlah nilai yang hilang (NaN) di setiap kolom DataFrame.

In [10]:
print(df.isnull().sum())

Article    0
Title      0
dtype: int64


## Menangani Duplikat
Tahapan ini Mengecek Apakah Ada Duplikat, dan bila ada maka duplikasi dihapus

In [11]:
df.drop_duplicates(inplace=True)
print(df.duplicated().sum())

0


## Melihat Sampel
Menampilkan 5 Sampel dari DataFrame

In [12]:
print(df.sample(5))

                                              Article  \
17  Clustering is the task of identifying similar ...   
6   Natural language processing or NLP is a subfie...   
14  Language detection is a natural language proce...   
12  API stands for Application Programming Interfa...   
22  Agglomerative clustering is based on hierarchi...   

                                           Title  
17     Clustering Algorithms in Machine Learning  
6                        Best Books to Learn NLP  
14      Language Detection with Machine Learning  
12          Best Python Frameworks to Build APIs  
22  Agglomerative Clustering in Machine Learning  


# 5. Preprocessing
Tahapan ini bertujuan membersihkan teks dari karakter tidak penting,
menghapus stopwords, dan melakukan stemming untuk mendapatkan bentuk kata dasar.
Gabungan antara kolom judul dan isi artikel digunakan sebagai basis fitur teks.
"""

In [13]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    text = str(text).lower()  # ubah ke huruf kecil
    text = re.sub(r'\d+', '', text)  # hapus angka
    text = text.translate(str.maketrans('', '', string.punctuation))  # hapus tanda baca
    tokens = text.split()
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]  # hapus stopwords dan stemming
    return ' '.join(tokens)

# Menggabungkan judul dan isi artikel
menggabungkan kolom `'Titles'` dan `'Article'` menjadi kolom `'text'`. Kemudian, membersihkan teks di kolom `'text'` menggunakan fungsi preprocess dan menyimpannya di kolom `'text_clean'`.

In [14]:
df['text'] = df['Title'] + " " + df['Article']
df['text_clean'] = df['text'].apply(preprocess)

# 6. Feature Extraction
 Melakukan Ektraksi Fitur, TF-IDF (Term Frequency-Inverse Document Frequency) digunakan untuk mengubah teks menjadi vektor numerik
 berdasarkan frekuensi kemunculan kata dalam dokumen relatif terhadap seluruh korpus.

In [15]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['text_clean'])

# 7. Similarity Calculation
Cosine similarity digunakan untuk mengukur tingkat kemiripan antar dua artikel berdasarkan vektor TF-IDF.

Nilainya berkisar antara 0 (tidak mirip) hingga 1 (sangat mirip).

In [16]:
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# 8. Rekomendasi Artikel
Fungsi get_recommendations menerima indeks artikel dan mengembalikan daftar top-n artikel paling mirip berdasarkan skor cosine similarity. Artikel input akan dilewati dalam hasil.

In [17]:
def get_recommendations(index, top_n=5):
    sim_scores = list(enumerate(cos_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]  # Skip artikel itu sendiri
    article_indices = [i[0] for i in sim_scores]
    return df.iloc[article_indices][['Title']]

# Contoh rekomendasi
article_id = 3
print("Artikel utama:")
print(df.iloc[article_id]['Title'])
print("\nRekomendasi artikel:")
print(get_recommendations(article_id))


Artikel utama:
Multiclass Classification Algorithms in Machine Learning

Rekomendasi artikel:
                                                Title
17          Clustering Algorithms in Machine Learning
31  Use Cases of Different Machine Learning Algori...
2           News Classification with Machine Learning
28                  Best Books to Learn Deep Learning
1          Assumptions of Machine Learning Algorithms


# 9. Evaluasi
Evaluasi dilakukan dengan cara observasi manual terhadap hasil rekomendasi artikel karena dataset tidak menyediakan label relevansi atau interaksi pengguna.

Artikel masukan: "Multiclass Classification Algorithms in Machine Learning"

Rekomendasi yang dihasilkan:

1. Clustering Algorithms in Machine Learning

2. Use Cases of Different Machine Learning Algorithms

3. News Classification with Machine Learning

4. Best Books to Learn Deep Learning

5. Assumptions of Machine Learning Algorithms

Analisis:

* Rekomendasi yang diberikan memiliki topik yang cukup relevan dengan artikel utama.

* Beberapa artikel membahas algoritma klasifikasi, penggunaan machine learning, serta pembelajaran mendalam (deep learning).

* Hal ini menunjukkan bahwa sistem mengenali kemiripan tema dan topik meskipun tidak menggunakan metadata atau label eksplisit.

Keterbatasan:

* Karena tidak ada label relevansi atau interaksi pengguna, tidak bisa dihitung metrik seperti Precision@k atau Recall.

* Rekomendasi bersifat statis, belum mempertimbangkan preferensi atau histori pengguna.

Kesimpulan Evaluasi:

1. Rekomendasi yang diberikan cukup relevan dan kontekstual terhadap artikel sumber.

2. Evaluasi berbasis pengamatan ini cukup untuk memvalidasi pendekatan content-based filtering pada kasus ini.


# 10. Kesimpulan
Proyek ini berhasil membangun sistem rekomendasi artikel menggunakan pendekatan Content-Based Filtering
berdasarkan kemiripan konten antarartikel. Metode yang digunakan meliputi:

- Preprocessing teks: pembersihan teks, penghapusan stopwords, dan stemming.
- Ekstraksi fitur: TF-IDF Vectorizer digunakan untuk merepresentasikan artikel dalam bentuk vektor numerik.
- Penghitungan kemiripan: cosine similarity menghitung tingkat kemiripan antar artikel.

Hasil evaluasi menunjukkan bahwa sistem mampu menghasilkan rekomendasi artikel yang relevan secara tematik dengan artikel acuan. Sebagai contoh, untuk artikel "Multiclass Classification Algorithms in Machine Learning",
sistem berhasil merekomendasikan artikel lain yang membahas clustering, penggunaan machine learning, dan deep learning,
yang masih dalam ranah keilmuan yang sama.

Kelebihan:
- Tidak memerlukan data interaksi pengguna.

- Efektif pada kasus dengan teks deskriptif yang kaya informasi.

Keterbatasan:
- Rekomendasi bersifat statis dan belum mempertimbangkan preferensi pengguna secara personal.

- Tidak tersedia metrik kuantitatif untuk evaluasi (seperti precision@k), karena dataset tidak menyertakan label relevansi.

Rekomendasi Pengembangan:
- Menggabungkan pendekatan ini dengan metode collaborative filtering jika data pengguna tersedia.

- Menyediakan antarmuka input berdasarkan judul artikel untuk kenyamanan pengguna.

Dengan pendekatan content-based ini, sistem rekomendasi dapat digunakan sebagai solusi awal yang sederhana namun efektif
untuk menyajikan artikel serupa kepada pembaca berdasarkan isi kontennya.
